In [27]:
!pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import scipy.stats as ss
from collections import Counter
import statsmodels.api as sm

In [2]:
df_gold = pd.read_csv('../data/mct_combined_v3.csv')
categories = ['Overall'] + list(set(df_gold['question_category']))
# dfx = pd.read_csv('../data/tiebreaking_needed.csv')
# df_gold = df_gold[~df_gold['Question'].isin(dfx['Question_x'])]
print(df_gold.shape[0])

1621


In [3]:
Counter(df_gold['question_category'])

Counter({'counseling skills and interventions': 600,
         'intake, assessment, and diagnosis': 466,
         'professional practice and ethics': 275,
         'treatment planning': 254,
         'core counseling attributes': 23,
         nan: 2,
         'counseling': 1})

In [29]:
df = pd.read_csv('./CounselingQA_metrics_combined.csv')
# dx = df[['setting', 'Accuracy_Overall', 'F1_Overall', 'Precision_Overall', 'Recall_Overall']]
# dx = dx[dx['setting'].str.contains('zero-shot_None_0')]
# dx.to_csv('./table1a.csv', index=False)
examined_models = ['Meta-Llama-3-70B-Instruct_few-shot_None_3', 'Llama3-Med42-70B_few-shot_None_3', 'Llama3-OpenBioLLM-70B_few-shot_None_3', 'counselingQA-gpt4o_zero-shot_None_0']
df = df[df['setting'].isin(examined_models)]
df.to_csv('./table2.csv', index=False)
# df = df[(df['setting'].str.contains('zero-shot_None_0')) | (df['setting'].str.contains('few-shot_None_3')) | (df['setting'].str.contains('zero-shot_sc_0')) | (df['setting'].str.contains('few-shot_sc_3')) | (df['setting'].str.contains('few-shot-cot_None_3'))]
# df['model_name'] = [x[:x.find('_')] for x in df['setting']]
# dfx = df[df['Accuracy_Overall'] > 0.6]
# df = df[df['model_name'].isin(dfx['model_name'])]
# df1 = df[df['setting'].str.contains('zero-shot_None_0')].sort_values(by=['setting'])
# df2 = df[df['setting'].str.contains('zero-shot_sc_0')].sort_values(by=['setting'])
# df3 = df[df['setting'].str.contains('few-shot_None_3')].sort_values(by=['setting'])
# df4 = df[df['setting'].str.contains('few-shot_sc_3')].sort_values(by=['setting'])
# df5 = df[df['setting'].str.contains('few-shot-cot_None_3')].sort_values(by=['setting'])
# df = pd.concat([df1, df2, df3, df4, df5], axis=False)
# df.to_csv('./table3.csv', index=False)

In [5]:
for r in df['setting']:
    print(r)

Asclepius-13B_few-shot_sc_1
medalpaca-7b_few-shot_None_1
Llama-2-13b-hf_few-shot_None_1
Asclepius-13B_few-shot_sc_3
BioMedGPT-LM-7B_few-shot-cot_None_3
Llama3-Med42-8B_few-shot_sc_1
Meta-Llama-3-8B-Instruct_few-shot_sc_1
Mistral-7B-Instruct-v0.3_few-shot-cot_None_1
BioMedGPT-LM-7B_few-shot_None_3
Llama3-Med42-70B_few-shot_None_5
MentaLLaMA-chat-13B_zero-shot_None_0
Llama3-Med42-8B_few-shot_sc_5
BioMedGPT-LM-7B_few-shot_sc_3
meditron-70b_few-shot-cot_None_1
Asclepius-7B_few-shot-cot_None_3
med42-70b_few-shot_None_3
Llama3-Med42-8B_few-shot_None_5
ClinicalCamel-70B_few-shot-cot_None_3
Llama3-Med42-70B_few-shot_sc_1
Llama-2-70b-hf_few-shot_None_1
Llama3-OpenBioLLM-8B_few-shot_None_3
ClinicalCamel-70B_few-shot_None_3
Llama-2-70b-hf_few-shot_None_3
ClinicalCamel-70B_zero-shot_None_0
meditron-70b_zero-shot_sc_0
meditron-7b_few-shot_sc_1
Llama-2-70b-hf_zero-shot_None_0
BioMedGPT-LM-7B_few-shot_sc_1
BioMedGPT-LM-7B_few-shot_sc_5
Llama-2-13b-chat-hf_zero-shot_sc_0
medalpaca-13b_few-shot_sc_5
me

In [32]:
model_list = []
for row in df.iterrows():
    setting = row[1]['setting']
    model_list.append(setting[:setting.find('_')])
model_set = set(model_list)
print(model_set)

{'MentaLLaMA-chat-13B', 'counselingQA-gpt4o', 'Llama-2-13b-hf', 'Llama-2-13b-chat-hf', 'Llama3-OpenBioLLM-70B', 'Llama3-Med42-70B', 'llama-2-7b-chat-hf', 'medalpaca-7b', 'Mistral-7B-Instruct-v0.3', 'Asclepius-Llama3-8B', 'Llama-2-70b-hf', 'Asclepius-7B', 'BioMedGPT-LM-7B', 'ClinicalCamel-70B', 'med42-70b', 'medalpaca-13b', 'Llama-2-70b-chat-hf', 'Asclepius-13B', 'Llama3-OpenBioLLM-8B', 'meditron-7b', 'Llama3-Med42-8B', 'Meta-Llama-3-8B-Instruct', 'Meta-Llama-3-70B-Instruct', 'Llama-2-7b-hf', 'meditron-70b'}


In [33]:
for x in df['setting']:
    print(x)

Asclepius-13B_few-shot_sc_1
medalpaca-7b_few-shot_None_1
Llama-2-13b-hf_few-shot_None_1
Asclepius-13B_few-shot_sc_3
BioMedGPT-LM-7B_few-shot-cot_None_3
Llama3-Med42-8B_few-shot_sc_1
Meta-Llama-3-8B-Instruct_few-shot_sc_1
Mistral-7B-Instruct-v0.3_few-shot-cot_None_1
BioMedGPT-LM-7B_few-shot_None_3
Llama3-Med42-70B_few-shot_None_5
MentaLLaMA-chat-13B_zero-shot_None_0
Llama3-Med42-8B_few-shot_sc_5
BioMedGPT-LM-7B_few-shot_sc_3
meditron-70b_few-shot-cot_None_1
Asclepius-7B_few-shot-cot_None_3
med42-70b_few-shot_None_3
Llama3-Med42-8B_few-shot_None_5
ClinicalCamel-70B_few-shot-cot_None_3
Llama3-Med42-70B_few-shot_sc_1
Llama-2-70b-hf_few-shot_None_1
Llama3-OpenBioLLM-8B_few-shot_None_3
ClinicalCamel-70B_few-shot_None_3
Llama-2-70b-hf_few-shot_None_3
ClinicalCamel-70B_zero-shot_None_0
meditron-70b_zero-shot_sc_0
meditron-7b_few-shot_sc_1
Llama-2-70b-hf_zero-shot_None_0
BioMedGPT-LM-7B_few-shot_sc_1
BioMedGPT-LM-7B_few-shot_sc_5
Llama-2-13b-chat-hf_zero-shot_sc_0
medalpaca-13b_few-shot_sc_5
me

In [34]:
'meditron-70b_zero-shot_sc_0' in list(df['setting'])

True

In [35]:
# Model check 
for model in model_set:
    # Zero-shot
    if f"{model}_zero-shot_None_0" not in list(df['setting']):
        print(f"{model} does not have zero-shot results")
    # Zero-shot
    if model + '_few-shot_None_1' not in list(df['setting']) and model + '_few-shot_None_3' not in list(df['setting']):
        print(f"{model} does not have few-shot results")
    # Zero-shot
    if model + '_zero-shot_sc_0' not in list(df['setting']):
        print(f"{model} does not have zero-shot, self-consistency results")
    # Zero-shot
    if model + '_few-shot_sc_1' not in list(df['setting']) and model + '_few-shot_sc_3' not in list(df['setting']):
        print(f"{model} does not have few-shot, self-consistency results")
    # Zero-shot
    if f'{model}_few-shot-cot_None_1' not in list(df['setting']) and f'{model}_few-shot-cot_None_3' not in list(df['setting']):
        print(f"{model} does not have few-shot, chain-of-thought results")

counselingQA-gpt4o does not have zero-shot, self-consistency results
counselingQA-gpt4o does not have few-shot, self-consistency results
Mistral-7B-Instruct-v0.3 does not have few-shot results
Mistral-7B-Instruct-v0.3 does not have zero-shot, self-consistency results
Mistral-7B-Instruct-v0.3 does not have few-shot, self-consistency results
med42-70b does not have few-shot, chain-of-thought results
meditron-7b does not have few-shot, chain-of-thought results
Meta-Llama-3-8B-Instruct does not have few-shot, chain-of-thought results
Meta-Llama-3-70B-Instruct does not have zero-shot, self-consistency results
Meta-Llama-3-70B-Instruct does not have few-shot, self-consistency results


In [55]:
generalist_models = ['llama-2-7b-chat-hf', 'Llama-2-7b-hf', 'Llama-2-13b-hf', 'Llama-2-70b-chat-hf', 'Meta-Llama-3-8B-Instruct', 'Meta-Llama-3-8B-Instruct', 'Meta-Llama-3-70B-Instruct', 'Meta-Llama-3-8B-Instruct', 'Meta-Llama-3-70B-Instruct', 'Llama-2-13b-chat-hf', 'Llama-2-7b-hf']
medical_models = ['BioMedGPT-LM-7B', 'Asclepius-7B', 'Asclepius-13B', 'meditron-70b', 'Llama3-Med42-8B', 'Asclepius-Llama3-8B', 'Llama3-Med42-70B',  'Llama3-OpenBioLLM-8B', 'Llama3-OpenBioLLM-70B', 'MentaLLaMA-chat-13B', 'meditron-7b']
generalist_models_l = ['Llama-2-70b-hf', 'Llama-2-70b-hf', 'Meta-Llama-3-8B-Instruct', 'Meta-Llama-3-70B-Instruct', 'Llama-2-13b-chat-hf']
medical_models_l = ['meditron-70b', 'ClinicalCamel-70B', 'Llama3-Med42-8B', 'Llama3-Med42-70B', 'MentaLLaMA-chat-13B']

In [56]:
competencies = ['Overall', 'core counseling attributes', 'counseling skills and interventions', 'intake, assessment, and diagnosis', 'professional practice and ethics', 'treatment planning']

In [61]:
mode = 'few-shot_None_3'
pvalues = []
for comp in competencies:
    print('#######################')
    print(comp)
    generalist_f1 = []
    medical_f1 = []
    for model in generalist_models:
        full_setting = model + "_" + mode
        # print(full_setting)
        generalist_f1.append(df[df['setting'] == full_setting].iloc[0][f'Accuracy_{comp}'])
        # print(df[df['setting'] == full_setting].iloc[0]['Accuracy_counseling skills and interventions'])
    for model in medical_models:
        full_setting = model + "_" + mode
        # print(full_setting)
        medical_f1.append(df[df['setting'] == full_setting].iloc[0][f'Accuracy_{comp}'])
        # print(df[df['setting'] == full_setting].iloc[0]['Accuracy_counseling skills and interventions'])
    print(generalist_f1)
    print(medical_f1)
    pval = ss.ttest_rel(generalist_f1, medical_f1)
    print(pval)
    pvalues.append(pval.pvalue)

#######################
Overall
[0.4488000000000001, 0.3208999999999999, 0.4097, 0.6214999999999999, 0.6507999999999999, 0.6507999999999999, 0.7339, 0.6507999999999999, 0.7339, 0.5386, 0.3208999999999999]
[0.3839000000000001, 0.2303, 0.3046, 0.5931, 0.6392000000000001, 0.3710000000000001, 0.7003, 0.5759, 0.7249999999999999, 0.4760999999999999, 0.2405]
TtestResult(statistic=3.398470105270606, pvalue=0.0067885013409609545, df=10)
#######################
core counseling attributes
[0.3505999999999999, 0.4800999999999999, 0.6522, 0.4285, 0.6607999999999999, 0.6607999999999999, 0.6975000000000002, 0.6607999999999999, 0.6975000000000002, 0.4772, 0.4800999999999999]
[0.5381, 0.2634999999999999, 0.3527, 0.5604, 0.5702, 0.3896, 0.6123999999999999, 0.4276, 0.61, 0.5229, 0.2649]
TtestResult(statistic=2.073769173353322, pvalue=0.06487241623955775, df=10)
#######################
counseling skills and interventions
[0.4393000000000001, 0.3253999999999999, 0.4251, 0.6143, 0.6624000000000001, 0.662400

In [62]:
ss.false_discovery_control(pvalues, method='bh')

array([0.0203655 , 0.06487242, 0.0203655 , 0.02156176, 0.03255109,
       0.02552323])

In [60]:
# generalist_f1

In [41]:
# medical_f1